In [1]:
import pandas as pd
import json 
import gzip
import os
import re as regex
import progressbar
import pickle
import numpy as np

In [2]:
product_path = '../../../../src/data/product'
train_test_all_filtered_path = os.path.join(product_path, 'train_test_split/output_unfiltered_tables/large/after_manual_checking')
files_representation_train = [file for file in os.listdir(os.path.join(train_test_all_filtered_path,'train')) if file.endswith('.json.gz')]
turl_input_path = os.path.join(product_path, 'TURL/input')

In [24]:
# # # use clean tables to get table representation for TURL rewritten without progressbar
# train_representation=[]
# for zip_file in files_representation_train:
#     print('/{}'.format(zip_file))
#     df = pd.read_json(os.path.join(train_test_all_filtered_path,'train') + '/{}'.format(zip_file), compression='gzip', lines=True)
#     if ('description' in df.columns)== False: # check if description column given
#         df['description'] = ''
#     if ('name' in df.columns)== False: # check if description column given
#         df['name'] = x = df.page_url.apply(lambda x: x.split("/")[4])
#     table_representation = [] #empty list for table
#     table_representation.append(zip_file)#append table id
#     table_representation.append('')#append page title -> not relevant
#     table_representation.append('')#append wikipedia page id -> not given
#     table_representation.append('')#append information about entity -> product, not relevant since same for all tables
#     table_representation.append('')#append table caption -> not given
#     table_representation.append(['name','description']) #append headers -> not sure if we should do that
#     all_rows_representation = []#representation of all rows
#     column_1_representation=[] # cell representation of column 1    
#     column_2_representation=[] # cell representation of column 1
#     for i in range(len(df)):
#         column_1_representation.append([[i,0],[df['row_id'][i],str(df['name'][i])]])
#         column_2_representation.append([[i,1],[df['row_id'][i],str(df['description'][i])]])
#     all_rows_representation.append(column_1_representation)
#     all_rows_representation.append(column_2_representation) #append single column representation to representation of all rows
#     table_representation.append(all_rows_representation)#append it to representation of whole table
#     table_representation.append(df['cluster_id'].apply(lambda x: [str(x)]).to_list())
#     train_representation.append(table_representation)

In [ ]:
# # use clean tables to get table representation for TURL rewritten
# count = 0
# train_representation=[] #representation of whole training set
# with progressbar.ProgressBar(max_value=len(files_representation_train)) as bar:
#     for zip_file in files_representation_train:
#         print('/{}'.format(zip_file))
#         df = pd.read_json(os.path.join(train_test_all_filtered_path,'train') + '/{}'.format(zip_file), compression='gzip', lines=True)
#         if ('description' in df.columns)== False: # check if description column given
#             df['description'] = ''
#         if ('name' in df.columns)== False: # check if description column given
#             df['name'] = x = df.page_url.apply(lambda x: x.split("/")[4])
#         table_representation = [] #empty list for table
#         table_representation.append(zip_file)#append table id
#         table_representation.append('')#append page title -> not relevant
#         table_representation.append('')#append wikipedia page id -> not given
#         table_representation.append('')#append information about entity -> product, not relevant since same for all tables
#         table_representation.append('')#append table caption -> not given
#         table_representation.append(['name','description']) #append headers -> not sure if we should do that
#         all_rows_representation = []#representation of all rows
#         for i in range(len(df)):
#             column_1_representation=[] # cell representation of column 1
#             column_2_representation=[] # cell representation of column 1
#             column_1_representation.append([[i,0],[df['row_id'][i],str(df['name'][i])]])
#             column_2_representation.append([[i,1],[df['row_id'][i],str(df['description'][i])]])
#         all_rows_representation.append(column_1_representation).append(column_2_representation) #append single column representation to representation of all rows
#         table_representation.append(all_rows_representation)#append it to representation of whole table
#         table_representation.append(df['cluster_id'].apply(lambda x: [str(x)]).to_list())
#         train_representation.append(table_representation)
#         count += 1
#         bar.update(count)

In [29]:
# # use clean tables to get table representation for TURL with transposed matrix
# train_representation=[] #representation of whole training set
# for zip_file in files_representation_train:
#     print('/{}'.format(zip_file))
#     df = pd.read_json(os.path.join(train_test_all_filtered_path,'val') + '/{}'.format(zip_file), compression='gzip', lines=True)
#     if ('description' in df.columns)== False: # check if description column given
#         df['description'] = ''
#     if ('name' in df.columns)== False: # check if description column given
#         df['name'] = x = df.page_url.apply(lambda x: x.split("/")[4])
#     df['header']='Product'#empty string for headers
#     table_representation = [] #empty list for table
#     table_representation.append(zip_file)#append table id
#     table_representation.append('')#append page title -> not relevant
#     table_representation.append('')#append wikipedia page id -> not given
#     table_representation.append('')#append information about entity -> product, not relevant since same for all tables
#     table_representation.append('')#append table caption -> not given
#     table_representation.append(df['header'].to_list()) #append headers -> not sure if we should do that
#     all_rows_representation = []#representation of all rows
#     for i in range(len(df)):
#         row_representation=[] # cell representation of single row
#         row_representation.append([[0,i],[df['row_id'][i],str(df['name'][i])]])
#         row_representation.append([[1,i],[df['row_id'][i],str(df['description'][i])]])
#         all_rows_representation.append(row_representation) #append single cell representation to representation of all rows
#     table_representation.append(all_rows_representation)#append it to representation of whole table
#     table_representation.append(df['cluster_id'].apply(lambda x: [str(x)]).to_list())
#     train_representation.append(table_representation)

/Product_lojapeterpaiva.com.br_September2020.json.gz
/Product_bspbikes.com_September2020.json.gz
/Product_northcentralcyclery.com_September2020.json.gz
/Product_grandrivercycle.com_September2020.json.gz
/Product_cyclesgalleria.com.au_September2020.json.gz
/Product_expotronica.com_September2020.json.gz
/Product_carpetoutletpelham.com_September2020.json.gz
/Product_carolinatriathlon.com_September2020.json.gz
/Product_sprocketandgear.co.uk_September2020.json.gz
/Product_artemisoverland.com_September2020.json.gz
/Product_mervisdiamond.com_September2020.json.gz
/Product_sheelys.com_September2020.json.gz
/Product_cartridges-directuk.com_September2020.json.gz
/Product_ubuy.com.kw_September2020.json.gz
/Product_salaostore.com.br_September2020.json.gz
/Product_zeuscomics.com_September2020.json.gz
/Product_grandoptical.com_September2020.json.gz
/Product_laptopsupermarket.com.au_September2020.json.gz
/Product_lablaab.com_September2020.json.gz
/Product_lovecycle.org_September2020.json.gz
/Product_

/Product_acmebicycles.com_September2020.json.gz
/Product_ballinacamerashop.com.au_September2020.json.gz
/Product_debeste.com_September2020.json.gz
/Product_gardenmachinerydirect.co.uk_September2020.json.gz
/Product_larkgifts.com_September2020.json.gz
/Product_jerseyvillecarpetfurniture.net_September2020.json.gz
/Product_bassappliance.com_September2020.json.gz
/Product_zipjosh.com_September2020.json.gz
/Product_bol.com_September2020.json.gz
/Product_trailshop.com_September2020.json.gz
/Product_headset-store.co.uk_September2020.json.gz
/Product_bunsieck.com_September2020.json.gz
/Product_bejattos.com_September2020.json.gz
/Product_localhoneyill.com_September2020.json.gz
/Product_coastallifetherapeutics.com_September2020.json.gz
/Product_claudettefloyd.com_September2020.json.gz
/Product_nicholsandsons.com_September2020.json.gz
/Product_zvejotribuna.net_September2020.json.gz
/Product_markagoo.com_September2020.json.gz
/Product_allface.com.br_September2020.json.gz
/Product_8storeytree.com_S

/Product_altex.com_September2020.json.gz
/Product_missnumerique.com_September2020.json.gz
/Product_northwestbicycle.com_September2020.json.gz
/Product_cyclepathhayward.com_September2020.json.gz
/Product_extra.com_September2020.json.gz
/Product_woodysfurnitureraton.com_September2020.json.gz
/Product_thehubpisgah.com_September2020.json.gz
/Product_houseofcardoon.com_September2020.json.gz
/Product_thebikerschoice.com_September2020.json.gz
/Product_hpe.com_September2020.json.gz
/Product_procyclingwarehouse.com_September2020.json.gz
/Product_fitchburgcycles.com_September2020.json.gz
/Product_dymocks.com.au_September2020.json.gz
/Product_adfflooring.com_September2020.json.gz
/Product_bensonappliance.com_September2020.json.gz
/Product_skinessentialsbeautybar.com_September2020.json.gz
/Product_mimaos.com_September2020.json.gz
/Product_sicarriere.com_September2020.json.gz
/Product_pc-ricondizionati.com_September2020.json.gz
/Product_thebikelabokc.com_September2020.json.gz
/Product_ussailing.org

/Product_barkermountainbikes.com_September2020.json.gz
/Product_mrjewelersonline.com_September2020.json.gz
/Product_bosssupplements.com_September2020.json.gz
/Product_arcade-zone.net_September2020.json.gz
/Product_graphic-reseau.com_September2020.json.gz
/Product_itienda-bolivia.com_September2020.json.gz
/Product_vittorioformalwear.com_September2020.json.gz
/Product_johnstonshomecenter.com_September2020.json.gz
/Product_waz.com.br_September2020.json.gz
/Product_brandonbikeworks.com_September2020.json.gz
/Product_fanthome.com_September2020.json.gz
/Product_rodatel.com_September2020.json.gz
/Product_thuddo.com_September2020.json.gz
/Product_buddalicious.net_September2020.json.gz
/Product_paraapothek.com_September2020.json.gz
/Product_southgeorgiafloors.com_September2020.json.gz
/Product_elitehomestore.com_September2020.json.gz
/Product_sistarecarpets.com_September2020.json.gz
/Product_actionwheels.com_September2020.json.gz
/Product_bicycleworldonline.com_September2020.json.gz
/Product_al

/Product_pcmegabits.com_September2020.json.gz
/Product_shop4se.com_September2020.json.gz
/Product_milhflor.com_September2020.json.gz
/Product_urbillig.com_September2020.json.gz
/Product_cyclesetsports.com_September2020.json.gz
/Product_envisioncleveland.org_September2020.json.gz
/Product_georgesmusic.com_September2020.json.gz
/Product_spindriftcyclesports.com_September2020.json.gz
/Product_boylesflooring.com_September2020.json.gz
/Product_samiwatches.com_September2020.json.gz
/Product_aus-cameras.com.au_September2020.json.gz
/Product_havanasunglasses.com.au_September2020.json.gz
/Product_gustafsonfurniture.com_September2020.json.gz
/Product_2techshop.com_September2020.json.gz
/Product_kechiq.com_September2020.json.gz
/Product_cheerstn.com_September2020.json.gz
/Product_maytomay.com_September2020.json.gz
/Product_jordandebut.com_September2020.json.gz
/Product_dreampc.com.au_September2020.json.gz
/Product_towpathbike.com_September2020.json.gz
/Product_villagepeddler.com_September2020.jso

/Product_wacocarpet.com_September2020.json.gz
/Product_blbikeshop.com_September2020.json.gz
/Product_compusa.com_September2020.json.gz
/Product_carpetconnectionpc.com_September2020.json.gz
/Product_hereketet.com_September2020.json.gz
/Product_zsyst.com_September2020.json.gz
/Product_go-sport.com_September2020.json.gz
/Product_musiclab.com.au_September2020.json.gz
/Product_duylinhcomputer.com_September2020.json.gz
/Product_marshallpierce.com_September2020.json.gz
/Product_malmquistflooring.com_September2020.json.gz
/Product_lookingglassphoto.com_September2020.json.gz
/Product_clubpremier.com_September2020.json.gz
/Product_castellostore.com_September2020.json.gz
/Product_sweetgrassbakery.com_September2020.json.gz
/Product_studiocare.com_September2020.json.gz
/Product_tecsoftcomputer.com.pe_September2020.json.gz
/Product_dutasarana.com_September2020.json.gz
/Product_adriamarine.net_September2020.json.gz
/Product_topshotworldwide.com_September2020.json.gz
/Product_professionalvideostore.co

/Product_watchobsession.co.uk_September2020.json.gz
/Product_zingbasket.com_September2020.json.gz
/Product_superiorsoundnwt.com_September2020.json.gz
/Product_thegpsstore.com_September2020.json.gz
/Product_buywisemall.com_September2020.json.gz
/Product_ugurfoto.com_September2020.json.gz
/Product_nutritioncentralusa.com_September2020.json.gz
/Product_veroscrap.com_September2020.json.gz
/Product_superco.com_September2020.json.gz
/Product_somdkitchenbathfloors.com_September2020.json.gz


In [30]:
with open(os.path.join(turl_input_path, 'val_representation_header_product.txt'), "wb") as fp:  #Pickling
    pickle.dump(train_representation, fp)

In [31]:
files_representation_train = [file for file in os.listdir(os.path.join(train_test_all_filtered_path,'val')) if file.endswith('.json.gz')]
with open(os.path.join(turl_input_path, 'val_representation_header_product.txt'), "rb") as fp:   # Unpickling
    train_representation = pickle.load(fp)
with open(os.path.join(turl_input_path, 'val_representation_header_product_small.txt'), "wb") as fp:  #Pickling
    pickle.dump(train_representation[:10], fp)

In [6]:
# # generate type_vocab.txt
# df_train= pd.read_json(os.path.join(train_test_all_filtered_path,'train/concatenated_data/train_all_filtered_tables.json.gz'), compression='gzip', lines=True)
# df_test= pd.read_json(os.path.join(train_test_all_filtered_path,'test/concatenated_data/test_all_filtered_tables.json.gz'), compression='gzip', lines=True)
# df_val= pd.read_json(os.path.join(train_test_all_filtered_path,'val/concatenated_data/val_all_filtered_tables.json.gz'), compression='gzip', lines=True)

In [15]:
# ids = df_train.cluster_id.astype(str).unique()
# ids = np.append(ids,'-100')
# len(ids)

1411

In [16]:
#save list of cluster ids to txt file
#pd.DataFrame(ids).to_csv(path_or_buf=os.path.join(turl_input_path, 'type_vocab_clusters.txt'),sep='\t', index=True, header=False)

In [14]:
#try to save data as json
files_representation_train = [file for file in os.listdir(os.path.join(train_test_all_filtered_path,'val')) if file.endswith('.json.gz')]
with open(os.path.join(turl_input_path, 'val_representation_header_product.txt'), "rb") as fp:   # Unpickling
    train_representation = pickle.load(fp)

In [15]:
import json
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)
jsonStr = json.dumps(train_representation, cls=NpEncoder)

In [16]:
with open(os.path.join(turl_input_path, 'val_representation_header_product.json'), 'w') as f:
    json.dump(jsonStr, f)
    

In [17]:
with open(os.path.join(turl_input_path, 'val_representation_header_product.json'), "r") as read_file:
    train_representation = json.load(read_file)